In [2]:
import time
import numpy as np
import random
import re
from typing import List, Tuple
import pandas as pd

In [3]:
# naive deck generation

def gen_function(seed, num_run):
    rcards = ['1'] * 26 
    bcards = ['0'] * 26
    cards = rcards+bcards # total 52 cards
    total = []

    for gamenum in range(num_run):
        random.seed(seed)
        random.shuffle(cards)
        #print(cards)
        total.append(''.join(cards))
    return total

In [4]:
def game_function(deck, p1_seq, p2_seq):
    outcomes = []
    stack = [] # to keep track of the cards
    p1_num_tricks = 0
    p2_num_tricks = 0
    p1_num_cards = 0
    p2_num_cards = 0
    
    for card in deck:
        stack.append(card)
        curstack = ''.join(stack[-3:])
        if curstack == p1_seq:
            p1_num_tricks += 1
            p1_num_cards += len(stack)
            stack = []
        elif curstack == p2_seq:
            p2_num_tricks += 1
            p2_num_cards += len(stack)
            stack = []
    return p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards

In [5]:
# base pseudocode
def whole_simulation(seed, num_run):
    i = 0
    game_df = pd.DataFrame(columns = ['seed', 'deck', 'p1_seq', 'p2_seq', 'p1_num_tricks', 'p2_num_tricks', 'p1_num_cards', 'p2_num_cards'])
    decks = gen_function(seed, num_run)
    for deck in decks:
        for p1_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
            for p2_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
                p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards = game_function(deck, p1_seq, p2_seq)
                game_df.loc[i] = [seed, deck, p1_seq, p2_seq, p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards]
                i += 1
    return game_df

In [59]:
import random
import numpy as np
import sqlite3
import pandas as pd

In [60]:
import penney_db1
from importlib import reload
reload(penney_db1);
from  penney_db1 import DB

In [61]:
simulation = whole_simulation(1, 10)

In [62]:
simulation.head()

,seed,deck,p1_seq,p2_seq,p1_num_tricks,p2_num_tricks,p1_num_cards,p2_num_cards
0,1,0101101101000110010010011001011110100101110000...,000,000,2,0,45,0
1,1,0101101101000110010010011001011110100101110000...,000,001,2,6,20,32
2,1,0101101101000110010010011001011110100101110000...,000,010,1,7,6,39
3,1,0101101101000110010010011001011110100101110000...,000,011,2,6,9,39
4,1,0101101101000110010010011001011110100101110000...,000,100,0,8,0,51


In [63]:
def make_database(simulation):
    db = DB()
    db.connect_db()
    db.insert_results(simulation)
    return db.get_database_file()

In [64]:
database = make_database(simulation)

In [69]:
def probabilities(database, ):
    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    
    sql1 ='''
    CREATE VIEW win_results AS
    SELECT 
        p1_seq,
        p2_seq,
        p1_num_cards,
        p2_num_cards,
        p1_num_tricks,
        p2_num_tricks,
        deck,
        CASE 
            WHEN p1_num_cards > p2_num_cards THEN 'p1'
            WHEN p1_num_cards < p2_num_cards THEN 'p2'
            ELSE 'draw'
    END AS win_cards,
    CASE 
        WHEN p1_num_tricks > p2_num_tricks THEN 'p1'
        WHEN p1_num_tricks < p2_num_tricks THEN 'p2'
        ELSE 'draw'
    END AS win_tricks
    FROM 
    results;
    '''
    db.run_action(sql1)
    sql2 = '''
    WITH TricksWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_tricks = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ), 
    CardWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_cards = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ),
    TimesPlayed AS (
        SELECT 
            p1_seq, 
            p2_seq, 
            COUNT(*) AS TimesPlayed
        FROM 
            win_results
        GROUP BY 
            p1_seq, 
            p2_seq
    )
    
    SELECT 
        t.p1_seq, 
        t.p2_seq, 
        IFNULL(t.p1_wins_by_tricks, 0) AS P1TrickWinCount,
        IFNULL(c.p1_wins_by_cards, 0) AS P1CardWinCount,
        tp.TimesPlayed, 
        IFNULL(1.0 * t.p1_wins_by_tricks / tp.TimesPlayed, 0) AS P1ProbWinTricks,
        IFNULL(1.0 * c.p1_wins_by_cards / tp.TimesPlayed, 0) AS P1ProbWinCard
    FROM 
        TricksWins t
    LEFT JOIN 
        CardWins c ON t.p1_seq = c.p1_seq AND t.p2_seq = c.p2_seq
    LEFT JOIN 
        TimesPlayed tp ON t.p1_seq = tp.p1_seq AND t.p2_seq = tp.p2_seq;
    '''
    prob_df = db.run_query(sql2)
    
    prob_p1card=prob_df['P1ProbWinCard'].values
    prob_p1trick=prob_df['P1ProbWinTrick'].values
    prob_p1card=np.reshape(prob_df['P1ProbWinCard'].values, (8,8))
    prob_p1trick=np.reshape(prob_df['P1ProbWinTrick'].values, (8,8))

    return prob_p1card, prob_p1trick

NameError: name 'results' is not defined

In [ ]:
db.get_database_file()

In [11]:
db = DB()

In [12]:
db.connect_db()

In [13]:
db.insert_results(simulation)

In [14]:
sql ='''
SELECT * FROM results
;'''
db.run_query(sql)

,seed,deck,p1_seq,p2_seq,p1_num_tricks,p2_num_tricks,p1_num_cards,p2_num_cards
0,1,0101101101000110010010011001011110100101110000...,000,000,2,0,45,0
1,1,0101101101000110010010011001011110100101110000...,000,001,2,6,20,32
2,1,0101101101000110010010011001011110100101110000...,000,010,1,7,6,39
3,1,0101101101000110010010011001011110100101110000...,000,011,2,6,9,39
4,1,0101101101000110010010011001011110100101110000...,000,100,0,8,0,51
...,...,...,...,...,...,...,...,...
635,1,0011111000010001110010011110010100001101101101...,111,011,1,7,3,44
636,1,0011111000010001110010011110010100001101101101...,111,100,3,6,12,39
637,1,0011111000010001110010011110010100001101101101...,111,101,3,5,26,23
638,1,0011111000010001110010011110010100001101101101...,111,110,3,5,23,25


In [15]:
sql1 ='''
CREATE VIEW win_results AS
SELECT 
    p1_seq,
    p2_seq,
    p1_num_cards,
    p2_num_cards,
    p1_num_tricks,
    p2_num_tricks,
    deck,
    CASE 
        WHEN p1_num_cards > p2_num_cards THEN 'p1'
        WHEN p1_num_cards < p2_num_cards THEN 'p2'
        ELSE 'draw'
    END AS win_cards,
    CASE 
        WHEN p1_num_tricks > p2_num_tricks THEN 'p1'
        WHEN p1_num_tricks < p2_num_tricks THEN 'p2'
        ELSE 'draw'
    END AS win_tricks
FROM 
    results;
'''
db.run_action(sql1)

In [16]:
sql2 =''' 
select distinct(deck) from win_results;'''
db.run_query(sql2)

,deck
0,0101101101000110010010011001011110100101110000...
1,0110000000110011101100110010010110010101111101...
2,1011101110101111001100100101001100001110001001...
3,1011111000000100100010001001011100111011100101...
4,0000110011111001001101010010010000110011110101...
5,1100011111101110111111000111001000000010001000...
6,1100011001010000010010011101001110111101100100...
7,1110000111011011101100111010000010110101010101...
8,0111101110001110000111100110001100100110011000...
9,0011111000010001110010011110010100001101101101...


In [161]:
sql1 = """
SELECT 
    all_combinations.p1_seq, 
    all_combinations.p2_seq, 
    COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks,
    (SELECT COUNT(DISTINCT deck) FROM win_results) AS total_distinct_decks 
FROM 
    (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
LEFT JOIN 
    win_results ON all_combinations.p1_seq = win_results.p1_seq 
                AND all_combinations.p2_seq = win_results.p2_seq
                AND win_results.win_tricks = 'p1'
GROUP BY 
    all_combinations.p1_seq, 
    all_combinations.p2_seq
"""
db.run_query(sql1)

,p1_seq,p2_seq,p1_wins_by_tricks,total_distinct_decks
0,000,000,10,10
1,000,001,3,10
2,000,010,2,10
3,000,011,1,10
4,000,100,0,10
...,...,...,...,...
59,111,011,0,10
60,111,100,2,10
61,111,101,2,10
62,111,110,4,10


In [24]:
sql1="""
WITH CardWins AS (
    SELECT 
        all_combinations.p1_seq, 
        all_combinations.p2_seq, 
        COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards
    FROM 
        (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
    LEFT JOIN 
        win_results ON all_combinations.p1_seq = win_results.p1_seq 
                    AND all_combinations.p2_seq = win_results.p2_seq
                    AND win_results.win_cards = 'p1'
    GROUP BY 
        all_combinations.p1_seq, 
        all_combinations.p2_seq
), TimesPlayed AS (
    SELECT 
        p1_seq, 
        p2_seq, 
        COUNT(*) AS TimesPlayed
    FROM 
        win_results
    GROUP BY 
        p1_seq, 
        p2_seq
)

SELECT 
    c.p1_seq, 
    c.p2_seq, 
    IFNULL(c.p1_wins_by_cards, 0) AS P1CardWinCount,
    t.TimesPlayed, 
    IFNULL(1.0 * c.p1_wins_by_cards / t.TimesPlayed, 0) AS P1ProbWinCard
FROM 
    CardWins c
LEFT JOIN 
    TimesPlayed t ON c.p1_seq = t.p1_seq AND c.p2_seq = t.p2_seq;
"""
banked_cards = db.run_query(sql1)

In [25]:
sql1="""
WITH TricksWins AS (
    SELECT 
        all_combinations.p1_seq, 
        all_combinations.p2_seq, 
        COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks
    FROM 
        (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
    LEFT JOIN 
        win_results ON all_combinations.p1_seq = win_results.p1_seq 
                    AND all_combinations.p2_seq = win_results.p2_seq
                    AND win_results.win_tricks = 'p1'
    GROUP BY 
        all_combinations.p1_seq, 
        all_combinations.p2_seq
), TimesPlayed AS (
    SELECT 
        p1_seq, 
        p2_seq, 
        COUNT(*) AS TimesPlayed
    FROM 
        win_results
    GROUP BY 
        p1_seq, 
        p2_seq
)

SELECT 
    c.p1_seq, 
    c.p2_seq, 
    IFNULL(c.p1_wins_by_tricks, 0) AS P1TrickWinCount,
    t.TimesPlayed, 
    IFNULL(1.0 * c.p1_wins_by_tricks / t.TimesPlayed, 0) AS P1ProbWinTricks
FROM 
    TricksWins c
LEFT JOIN 
    TimesPlayed t ON c.p1_seq = t.p1_seq AND c.p2_seq = t.p2_seq;
"""
banked_tricks = db.run_query(sql1)

In [26]:
banked_tricks_arr = banked_tricks.to_numpy() 
banked_cards_arr = banked_cards.to_numpy()

In [37]:
##retrieving all the probabilites of player 1 winning by card
all_pcts_c=[]
all_pcts_t=[]
for i in range(64):
    all_pcts_c+=[banked_cards_arr[i][-1]]
    all_pcts_t+=[banked_tricks_arr[i][-1]]
new_cards=np.reshape(all_pcts_c, (8,8))
new_tricks=np.reshape(all_pcts_t, (8,8))
print(new_cards)
print('\n' ,new_tricks)

[[1.  0.3 0.2 0.2 0.  0.2 0.  0.5]
 [0.7 1.  1.  0.9 0.  0.9 0.6 1. ]
 [0.8 0.  1.  0.4 0.3 0.8 0.  0.6]
 [0.8 0.  0.6 1.  0.3 0.5 1.  1. ]
 [1.  1.  0.6 0.7 1.  0.9 0.  0.9]
 [0.8 0.1 0.2 0.5 0.1 1.  0.1 0.5]
 [1.  0.4 0.9 0.  1.  0.9 1.  0.7]
 [0.4 0.  0.4 0.  0.1 0.4 0.3 1. ]]

 [[1.  0.3 0.2 0.1 0.  0.2 0.  0.1]
 [0.6 1.  0.9 0.8 0.1 0.9 0.4 1. ]
 [0.8 0.  1.  0.4 0.3 0.3 0.1 0.8]
 [0.9 0.  0.5 1.  0.2 0.4 0.9 1. ]
 [1.  0.9 0.6 0.7 1.  0.7 0.  0.8]
 [0.8 0.1 0.3 0.5 0.1 1.  0.  0.6]
 [1.  0.2 0.9 0.1 1.  0.8 1.  0.6]
 [0.5 0.  0.2 0.  0.2 0.2 0.4 1. ]]


In [49]:
db.close()